# Video Summarization Using KMeans Clustering

In [ ]:
import scipy.io
import numpy as np
import cv2
from sklearn.cluster import KMeans

The values below can be treated as input parameters. They represent the following:
- sampling rate = Every nth frame is chosen
- percent = length of the video summary
- skim length = length of sequences of frames

In [ ]:
sampling_rate=6#3
percent=100
skim_length=0.5

Read the video using OpenCV. Calculate the length of the skim based on the number of frames per second.

In [ ]:
video=cv2.VideoCapture(r'Air_Force_One.mp4')
fps=int(video.get(cv2.CAP_PROP_FPS))
frame_count=int(video.get(cv2.CAP_PROP_FRAME_COUNT))
skim_frames_length=fps*skim_length

Retrieve the required frames and store it into an array

In [ ]:
frames = []
i=0
while(video.isOpened()):
    if i%sampling_rate==0:
        video.set(1,i)
        ret, frame = video.read()
        if frame is None :
            break
        frames.append(np.asarray(frame))
    i+=1
frames = np.array(frames)

In [ ]:
print("Length of video",frames.shape)
rows,cols = frames.shape[1],frames.shape[2]

Retreive the required features using OpenCV Histogram

In [ ]:
def get_color_hist(frames_raw, num_bins):
    print ("Generate linear Histrograms using OpenCV")
    num_channels=3
    histogram=[]
    
    for frame in frames_raw:
        feature_value=[cv2.calcHist([frame],[i],None,[int(num_bins)],[0,256]) for i in num_channels)]
        histogram.append(np.asarray(feature_value).flatten())
    histogram=np.asarray(histogram)
    return histogram

Calculate the number of centroids. Apply KMeans clustering on the data.

In [ ]:
features=get_color_hist(frames,16)
print("Shape of features: ",features.shape)

# Choosing number of centers for clustering
num_centroids=int(percent*frame_count/skim_frames_length/100)+1
print("Number of clusters: "+str(num_centroids))
kmeans=KMeans(n_clusters=num_centroids).fit(features)

Retrieve all the relevant frames from each cluster

In [ ]:
centres=[]
features_transform=kmeans.transform(features)
for cluster in range(features_transform.shape[1]):
    centres.append(np.argmin(features_transform.T[cluster]))

centres=sorted(centres)
frames_indices=[]
for centre in centres:
    for idx in range(max(int(centre*sampling_rate-skim_frames_length/2),0),min(int(centre*sampling_rate+skim_frames_length/2)+1,frame_count)):
        frames_indices.append(idx)
frames_indices=sorted(set(frames_indices))

Visualize the Output

In [ ]:
out = cv2.VideoWriter('afo_out.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, (cols,rows))

In [ ]:
i=0
while(video.isOpened()):
    if i >max(frames_indices):
        out.release()
        video.release()
    if i in frames_indices:
        ret, frame = video.read()
        if frame is None :
            break
        out.write(frame)
    i+=1